In [1]:
import pandas as pd
import os
from datasets import load_dataset, load_from_disk, Audio

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sli_csv='/Users/markjos/projects/malachor5/data/tira-code-mixed-snippets/tira-cm-snip-sli.csv'

In [3]:
sli_df=pd.read_csv(sli_csv)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/markjos/projects/malachor5/data/tira-code-mixed-snippets/tira-cm-snip-sli.csv'

In [4]:
sli_df.columns

NameError: name 'sli_df' is not defined

Good news, all mother clip and daughter snippet files exist!

In [19]:
('/Users/markjos/projects/malachor5/data/tira-code-mixed/'+sli_df['mother_file_name']).apply(os.path.exists).value_counts()

mother_file_name
True    43719
Name: count, dtype: int64

In [25]:
('/Users/markjos/projects/malachor5/data/tira-code-mixed-snippets/clips/train/'+sli_df['audio_path']).apply(os.path.exists).value_counts()

audio_path
True    43719
Name: count, dtype: int64

Let's add a column for the index of a snippet within a clip

In [ ]:
sli_df['mother_start_list']=sli_df['mother_start_list'].apply(eval)

In [44]:
sli_df['mother_start_list']=sli_df['mother_start_list'].apply(lambda l:[int(x) for x in l])

In [62]:
sli_df['clip_i']=0
def set_clip_indices(mother_clip):
    brothers=sli_df[sli_df['mother_file_name']==mother_clip].copy()
    brothers=brothers.sort_values('start')
    sli_df.loc[brothers.index, 'clip_i']=list(range(len(brothers)))
    
pd.Series(sli_df['mother_file_name'].unique()).apply(set_clip_indices)

0       None
1       None
2       None
3       None
4       None
        ... 
4416    None
4417    None
4418    None
4419    None
4420    None
Length: 4421, dtype: object

In [70]:
sli_df.sort_values(['mother_file_name','clip_i'])[['start', 'mother_file_name', 'clip_i']].iloc[:20]

,start,mother_file_name,clip_i
9,195.193,clips/train/HH01082021-m03s07ms011-m04s09ms552...,0
0,198.584,clips/train/HH01082021-m03s07ms011-m04s09ms552...,1
1,200.559,clips/train/HH01082021-m03s07ms011-m04s09ms552...,2
2,204.440,clips/train/HH01082021-m03s07ms011-m04s09ms552...,3
3,207.275,clips/train/HH01082021-m03s07ms011-m04s09ms552...,4
4,209.705,clips/train/HH01082021-m03s07ms011-m04s09ms552...,5
5,212.034,clips/train/HH01082021-m03s07ms011-m04s09ms552...,6
6,212.675,clips/train/HH01082021-m03s07ms011-m04s09ms552...,7
7,217.011,clips/train/HH01082021-m03s07ms011-m04s09ms552...,8
8,221.371,clips/train/HH01082021-m03s07ms011-m04s09ms552...,9


Now let's condense English records using a while loop

Rip numpy array operations :(

In [74]:
sli_df.columns

Index(['start', 'end', 'mother_old_start', 'mother_old_end',
       'mother_transcription', 'mother_eaf_source', 'mother_wav_source',
       'mother_raw_transcription', 'mother_clip', 'mother_wav_rawpath',
       'mother_path', 'mother_allosaurus', 'mother_clap_ipa_cos_sim',
       'mother_wada_snr', 'mother_nist_stnr', 'mother_speaker-diarization-3.1',
       'mother_voice-activity-detection', 'mother_whisper-large-v3',
       'mother_clapipa-transcription-allosaurus', 'mother_vad_s',
       'mother_drz_s', 'mother_trans_len', 'mother_pcnt_speech',
       'mother_trans_len_sq', 'mother_trans_len_log', 'mother_cos_sim_softmax',
       'mother_cos_sim_log', 'mother_duration', 'mother_split', 'mother_start',
       'mother_end', 'mother_overlap', 'mother_num_overlap',
       'mother_overlap_split', 'mother_start_list', 'mother_end_list',
       'mother_trans_list', 'mother_pairwise_cos_sim', 'mother_60s_start',
       'mother_60s_end', 'mother_file_name', 'audio_path', 'output', 'clip_i'

In [ ]:
def condense_records(mother_file_name):
    record_df=sli_df[sli_df['mother_file_name']==mother_file_name]
    condensed_rows=[]
    i=0
    while i<len(record_df):
        row=record_df.iloc[i]
        start=row['start']
        start_i=row['i']
        i+=1
        if row['output']!=1:
            condensed_rows.append(row)
            continue
        while i<len(record_df) and record_df.iloc[i]['output']==1:
            row=record_df.iloc[i]
            end_i=row['end']
            i+=1
        new_row=row.copy()
        new_row['start']=start
        new_row=['i']=(start_i,end_i)
        condensed_rows.append(new_row)
